In [5]:
using XLSX, DataFrames, TimeSeries
using StatsBase
using Plots, StatsPlots
using LinearAlgebra
using GLM
using FredData
include("utils.jl")


print_percentiles (generic function with 1 method)

In [93]:
data = DataFrame(XLSX.readtable("assets_data_03.24.xlsx", "Macro_Data"; infer_eltypes=true))
data = TimeArray(data,timestamp=:Date)

210×15 TimeArray{Any, 2, Date, Matrix{Any}} 1971-10-01 to 2024-01-01
┌────────────┬───────────────┬─────────────────┬─────────────────┬──────────────
│            │ NGDPRSAXDCPLQ │ POLCPIALLQINMEI │ IRLTLT01PLM156N │ IR3TIB01PLQ ⋯
├────────────┼───────────────┼─────────────────┼─────────────────┼──────────────
│ 1971-10-01 │       missing │         missing │         missing │         mis ⋯
│ 1972-01-01 │       missing │         missing │         missing │         mis ⋯
│ 1972-04-01 │       missing │         missing │         missing │         mis ⋯
│ 1972-07-01 │       missing │         missing │         missing │         mis ⋯
│ 1972-10-01 │       missing │         missing │         missing │         mis ⋯
│ 1973-01-01 │       missing │         missing │         missing │         mis ⋯
│ 1973-04-01 │       missing │         missing │         missing │         mis ⋯
│ 1973-07-01 │       missing │         missing │         missing │         mis ⋯
│     ⋮      │       ⋮       │        ⋮ 

In [100]:
cpi = from(data[:,[:cpi_pl, :cpi_us, :cpi_ez]],Date(2001,01,01))
real_short = from(data[:,[:IR3TIB01PLQ156N, :IR3TIB01USQ156N, :IR3TIB01EZQ156N]],Date(2001,01,01))./100 .- cpi

TypeError: TypeError: non-boolean (Missing) used in boolean context

In [91]:
real_rates = 

MethodError: MethodError: no method matching skewness(::Vector{Any})

Closest candidates are:
  skewness(!Matched::Distributions.Chernoff)
   @ Distributions C:\Users\matsz\.julia\packages\Distributions\UaWBm\src\univariate\continuous\chernoff.jl:210
  skewness(!Matched::Distributions.DiscreteUniform)
   @ Distributions C:\Users\matsz\.julia\packages\Distributions\UaWBm\src\univariate\discrete\discreteuniform.jl:58
  skewness(!Matched::Distributions.Hypergeometric)
   @ Distributions C:\Users\matsz\.julia\packages\Distributions\UaWBm\src\univariate\discrete\hypergeometric.jl:66
  ...


In [13]:
inflation = percentchange(cpi,:log)


TypeError: TypeError: non-boolean (Missing) used in boolean context

In [72]:
log1p.(short_data[:,[:IRLTLT01PLM156N, :IR3TIB01PLQ156N, :IRLTLT01USQ156N, :IR3TIB01USQ156N, :IRLTLT01EZM156N, :IR3TIB01EZQ156N]] ./ 100)

93×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 2001-01-01 to 2024-01-01
┌────────────┬─────────────────┬─────────────────┬─────────────────┬────────────
│            │ IRLTLT01PLM156N │ IR3TIB01PLQ156N │ IRLTLT01USQ156N │ IR3TIB01U ⋯
├────────────┼─────────────────┼─────────────────┼─────────────────┼────────────
│ 2001-01-01 │       0.0991815 │        0.166982 │       0.0492662 │       0.0 ⋯
│ 2001-04-01 │         0.10595 │        0.155293 │       0.0513583 │       0.0 ⋯
│ 2001-07-01 │        0.110766 │        0.141123 │       0.0485997 │       0.0 ⋯
│ 2001-10-01 │       0.0899321 │        0.119855 │       0.0465973 │        0. ⋯
│ 2002-01-01 │       0.0797965 │       0.0989399 │       0.0495201 │       0.0 ⋯
│ 2002-04-01 │       0.0762818 │       0.0912717 │       0.0497421 │       0.0 ⋯
│ 2002-07-01 │       0.0696193 │       0.0792732 │       0.0417176 │       0.0 ⋯
│ 2002-10-01 │       0.0580802 │       0.0665677 │       0.0392848 │       0.0 ⋯
│     ⋮      │        ⋮        │  

In [80]:
cpi = short_data[:POLCPIALLQINMEI, :USACPIALLMINMEI, :CP0000EZ19M086NEST]


93×3 TimeArray{Any, 2, Date, Matrix{Any}} 2001-01-01 to 2024-01-01
┌────────────┬─────────────────┬─────────────────┬────────────────────┐
│            │ POLCPIALLQINMEI │ USACPIALLMINMEI │ CP0000EZ19M086NEST │
├────────────┼─────────────────┼─────────────────┼────────────────────┤
│ 2001-01-01 │          73.028 │         74.1297 │              76.72 │
│ 2001-04-01 │         74.3877 │         74.9032 │              77.82 │
│ 2001-07-01 │         74.3408 │         75.0016 │              77.87 │
│ 2001-10-01 │         74.7863 │         74.7907 │              78.14 │
│ 2002-01-01 │         75.6303 │         75.0579 │              78.68 │
│ 2002-04-01 │         75.9116 │         75.8736 │              79.46 │
│ 2002-07-01 │         75.2083 │         76.1971 │              79.51 │
│ 2002-10-01 │         75.4427 │         76.4361 │              79.94 │
│     ⋮      │        ⋮        │        ⋮        │         ⋮          │
│ 2022-07-01 │         133.768 │         125.062 │             118.08